In [1]:
import sys
sys.path.append("../..")
from thesis.utils.utils import setup_ray, save, load, Experiment
path = "D:/Master/Masterarbeit/thesis"
setup_ray(path = path, unidirectional = False, seed=69)

2023-04-21 19:28:12,858	INFO worker.py:1528 -- Started a local Ray instance.


In [2]:
env_args = dict(
    fleetsize = 8,
    #fleetsize_upper = 12,
    max_fleetsize = 20,    
    pseudo_routing = False,
    pseudo_dispatcher = True,
    pseudo_dispatcher_clever = False,
    #pseudo_dispatcher_distance = 0.3,
    routing_agent_death= True,
    death_on_target = False,
    transform_dispatching_partobs = True,
    direction_reward = -0.1,
    sim_config = dict(
        dispatch = True,
        routing_ma = True,
        dispatching_ma = True,
        reward_reached_target = 10,
        #reward_reached_target_by_time = True, 
        reward_wrong_target = -1,
        reward_removed_for_block = -5, 
        reward_target_distance = 0,
        reward_invalid= -0.1,
        reward_duration = -0.5,
        reward_pass = 0.2,
        block_timeout = 120,
        reward_accepted_in_station = 1,
        reward_declined_in_station = -1,
        routing_interval = 2,
        dispatching_interval=360,
        io_quote = 0.99,
        availability = 0.95,
        mttr = 5,
    )
)

In [3]:
agv_model = dict(
    model = dict(
        custom_model = "gnn_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=16,
            with_action_mask=False,
            with_agvs=True,
            with_stations = False,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
            env_type = "matrix",
            n_convolutions = 2
        )
    )
)
dispatcher_model = dict(
    model = dict(
        custom_model = "lin_model",
        #custom_action_dist="MAActionDistribution",
        custom_model_config = dict(
            embed_dim=64,
            with_action_mask=False,
            with_agvs=True,
            with_stations = True,
            position_embedd_dim = 0,
            ff_embedd_dim = 4,
        )
    )
)

In [4]:
exp = Experiment("matrix_routing")
for seed in [44]:
    exp.experiment(
        path = path,
        env_args = env_args, 
        agv_model = agv_model,
        dispatcher_model=dispatcher_model,
        run_name="08_width", 
        env = "matrix",
        algo = "dqn",
        n_intervals =8,
        #train_agv = False,
        backup_interval=50,
        batch_size=300, #apex + gnn: 50
        seed = seed,
        algo_params = {"grad_clip": 1, "gamma":0.9, "exploration_config":{"warmup_timesteps": 0,"epsilon_timesteps": 200000,"final_epsilon": 0.02,"initial_epsilon": 1.0,"type": "EpsilonGreedy"}},# "exploration_config": {"type": "Curiosity", "sub_exploration": {"type": "StochasticSampling"}, "eta": 0.1}},
        lr = 1e-3,
        #load_agv="../../models/matrix_routing/07_reduce_grad_8_20_2023-03-04_17-58-45/checkpoint_000400",
        
    )

2023-04-21 19:28:21,285	INFO simple_q.py:307 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
2023-04-21 19:28:21,295	INFO algorithm.py:457 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_81136_ptfp1km_)
  warn(f"Unzipping to temporary directory ({tmp_dir})")
2023-04-21 19:28:47,525	INFO trainable.py:164 -- Trainable.setup took 26.245 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_12692_c_kqmotv)
  

ValueError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 2.00 GiB total capacity; 1.28 GiB already allocated; 0 bytes free; 1.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
 tracebackTraceback (most recent call last):
  File "c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\ray\rllib\policy\torch_policy.py", line 1068, in _worker
    self._loss(self, model, self.dist_class, sample_batch)
  File "c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\ray\rllib\algorithms\dqn\dqn_torch_policy.py", line 280, in build_q_losses
    q_tp1, q_logits_tp1, q_probs_tp1, _ = compute_q_values(
  File "c:\Users\Wegma\.conda\envs\thesis3_9\lib\site-packages\ray\rllib\algorithms\dqn\dqn_torch_policy.py", line 429, in compute_q_values
    model_out, state = model(input_dict, state_batches or [], seq_lens)
  File "c:\Users\Wegma\.con

In [ ]:
#load(exp.trainer, "agv", "../../models/trained")

In [ ]:
#save(exp.trainer, "agv", "../../models/trained_routing_newmat")

In [ ]:
#load(exp.trainer, "agv", "../../models/trained_routing")

In [ ]:
#exp.keep_training(6)

In [ ]:
#exp.keep_training(10)